## Import Library

In [1]:
from machine_lib import * 
import random

## 1, Login
在machine_lib文件的login方法中填写用户名和密码后保存然后来到本文件Restart Kernal后重新import machine_lib

In [2]:
s = login()

b'{"user":{"id":"TJ91365"},"token":{"expiry":14400.0},"permissions":["CONSULTANT","MULTI_SIMULATION","PROD_ALPHAS","REFERRAL","VISUALIZATION","WORKDAY"]}'


## 2, get data fields

In [9]:
def process_datafields2(df, data_type):

    if data_type == "matrix":
        datafields = df[df['type'] == "MATRIX"]["id"].tolist()
    elif data_type == "vector":
        datafields = get_vec_fields(df[df['type'] == "VECTOR"]["id"].tolist())

    tb_fields = []
    for field in datafields:
        tb_fields.append(f"- {field} * ts_std_dev({field}, 20)")
        #tb_fields.append("winsorize(ts_backfill(%s, 120), std=4)"%field)
        #- assets * ts_std_dev(assets, 20)
    return tb_fields

In [4]:
def get_datasets(
    s,
    instrument_type: str = 'EQUITY',
    region: str = 'USA',
    delay: int = 1,
    universe: str = 'TOP3000'
):
    url = "https://api.worldquantbrain.com/data-sets?" +\
        f"instrumentType={instrument_type}&region={region}&delay={str(delay)}&universe={universe}"
    result = s.get(url)
    datasets_df = pd.DataFrame(result.json()['results'])
    return datasets_df

In [5]:
datasets_df = get_datasets(s) # by default we load all datasets USA TOP3000 delay 1
datasets_df.head() # DataFrame.head() shows first 5 rows of the dataframe 

,id,name,description,category,subcategory,region,delay,universe,coverage,valueScore,userCount,alphaCount,fieldCount,pyramidMultiplier,themes,researchPapers
0,analyst11,ESG scores,Environmental Social Governance scores that ex...,"{'id': 'analyst', 'name': 'Analyst'}","{'id': 'analyst-esg', 'name': 'ESG'}",USA,1,TOP3000,0.6818,3.0,321,3887,197,1.3,[],"[{'type': 'discussion', 'title': 'Getting star..."
1,analyst14,Estimations of Key Fundamentals,This dataset reports many items from financial...,"{'id': 'analyst', 'name': 'Analyst'}","{'id': 'analyst-analyst-estimates', 'name': 'A...",USA,1,TOP3000,0.6203,2.0,677,43791,868,1.3,[],"[{'type': 'discussion', 'title': 'Getting star..."
2,analyst15,Earnings forecasts,This dataset provides bottom-up forecast data ...,"{'id': 'analyst', 'name': 'Analyst'}","{'id': 'analyst-analyst-estimates', 'name': 'A...",USA,1,TOP3000,0.9892,2.0,858,75662,2538,1.3,[],"[{'type': 'discussion', 'title': 'Getting star..."
3,analyst16,Real Time Estimates,This dataset provides real-time access to the ...,"{'id': 'analyst', 'name': 'Analyst'}","{'id': 'analyst-crowdsourced-estimates', 'name...",USA,1,TOP3000,0.8252,3.0,217,3288,107,1.3,[],"[{'type': 'research', 'title': 'Creating D0 Al..."
4,analyst35,ESG Model,The dataset provide ESG related information ba...,"{'id': 'analyst', 'name': 'Analyst'}","{'id': 'analyst-esg', 'name': 'ESG'}",USA,1,TOP3000,0.3771,4.0,69,455,46,1.3,[],"[{'type': 'discussion', 'title': 'Getting star..."


In [6]:
datasets = datasets_df[datasets_df['alphaCount']>20000]['id'].tolist()
len(datasets)

15

In [7]:
datasets[:15]

['analyst14',
 'analyst15',
 'analyst4',
 'analyst7',
 'fundamental17',
 'fundamental2',
 'fundamental23',
 'fundamental28',
 'fundamental6',
 'model165',
 'model26',
 'model77',
 'news12',
 'option8',
 'pv1']

In [10]:
pc_fields = []
for dd in datasets:
    df = get_datafields(s, dataset_id = dd, region='USA', universe='TOP3000', delay=1)
    temp = process_datafields2(df, "matrix") + process_datafields2(df, "vector")
    pc_fields = pc_fields + temp

print(pc_fields[0])
len(pc_fields)

- anl14_actvalue_bvps_fp0 * ts_std_dev(anl14_actvalue_bvps_fp0, 20)


18601

In [11]:
len(pc_fields)

18601

## 3, Alpha factory
### start with First Order

In [12]:
first_order = get_first_order(pc_fields, ts_ops)
print(first_order[:10])
print(len(first_order))

['- anl14_actvalue_bvps_fp0 * ts_std_dev(anl14_actvalue_bvps_fp0, 20)', 'ts_rank(- anl14_actvalue_bvps_fp0 * ts_std_dev(anl14_actvalue_bvps_fp0, 20), 5)', 'ts_rank(- anl14_actvalue_bvps_fp0 * ts_std_dev(anl14_actvalue_bvps_fp0, 20), 22)', 'ts_rank(- anl14_actvalue_bvps_fp0 * ts_std_dev(anl14_actvalue_bvps_fp0, 20), 66)', 'ts_rank(- anl14_actvalue_bvps_fp0 * ts_std_dev(anl14_actvalue_bvps_fp0, 20), 120)', 'ts_rank(- anl14_actvalue_bvps_fp0 * ts_std_dev(anl14_actvalue_bvps_fp0, 20), 240)', 'ts_zscore(- anl14_actvalue_bvps_fp0 * ts_std_dev(anl14_actvalue_bvps_fp0, 20), 5)', 'ts_zscore(- anl14_actvalue_bvps_fp0 * ts_std_dev(anl14_actvalue_bvps_fp0, 20), 22)', 'ts_zscore(- anl14_actvalue_bvps_fp0 * ts_std_dev(anl14_actvalue_bvps_fp0, 20), 66)', 'ts_zscore(- anl14_actvalue_bvps_fp0 * ts_std_dev(anl14_actvalue_bvps_fp0, 20), 120)']
1599686


List the Alphas

In [13]:
# Pad initial decay with alpha
init_decay = 4
fo_alpha_list = []
for alpha in first_order:
    fo_alpha_list.append((alpha, init_decay))
print(len(fo_alpha_list))
print(fo_alpha_list[:5])

1599686
[('- anl14_actvalue_bvps_fp0 * ts_std_dev(anl14_actvalue_bvps_fp0, 20)', 4), ('ts_rank(- anl14_actvalue_bvps_fp0 * ts_std_dev(anl14_actvalue_bvps_fp0, 20), 5)', 4), ('ts_rank(- anl14_actvalue_bvps_fp0 * ts_std_dev(anl14_actvalue_bvps_fp0, 20), 22)', 4), ('ts_rank(- anl14_actvalue_bvps_fp0 * ts_std_dev(anl14_actvalue_bvps_fp0, 20), 66)', 4), ('ts_rank(- anl14_actvalue_bvps_fp0 * ts_std_dev(anl14_actvalue_bvps_fp0, 20), 120)', 4)]


In [14]:
# Create a new shuffled list
shuffled_alpha_list = fo_alpha_list[:]  # Create a copy of the original list
random.shuffle(shuffled_alpha_list)  # Shuffle the copy

# Print the shuffled list
print(len(shuffled_alpha_list))  # Length remains the same
print(shuffled_alpha_list[:5])  # First 5 elements (now shuffled)

1599686
[('ts_ir(- historical_volatility_60 * ts_std_dev(historical_volatility_60, 20), 22)', 4), ('ts_zscore(- anl15_cpsgics_gr_cal_fy3_pe * ts_std_dev(anl15_cpsgics_gr_cal_fy3_pe, 20), 5)', 4), ('ts_product(- fnd28_value_09121 * ts_std_dev(fnd28_value_09121, 20), 66)', 4), ('ts_max_diff(- fnd6_newqv1300_glced12 * ts_std_dev(fnd6_newqv1300_glced12, 20), 22)', 4), ('ts_ir(- news_ls * ts_std_dev(news_ls, 20), 240)', 4)]


In [15]:
fo_alpha_list = shuffled_alpha_list[:200000]

In [16]:
fo_alpha_list[0]

('ts_ir(- historical_volatility_60 * ts_std_dev(historical_volatility_60, 20), 22)',
 4)

In [17]:
# Load alphas to task pools
pools = load_task_pool(fo_alpha_list, 10, 9)
print(pools[0])

[[('ts_ir(- historical_volatility_60 * ts_std_dev(historical_volatility_60, 20), 22)', 4), ('ts_zscore(- anl15_cpsgics_gr_cal_fy3_pe * ts_std_dev(anl15_cpsgics_gr_cal_fy3_pe, 20), 5)', 4), ('ts_product(- fnd28_value_09121 * ts_std_dev(fnd28_value_09121, 20), 66)', 4), ('ts_max_diff(- fnd6_newqv1300_glced12 * ts_std_dev(fnd6_newqv1300_glced12, 20), 22)', 4), ('ts_ir(- news_ls * ts_std_dev(news_ls, 20), 240)', 4), ('ts_scale(- est_q_bps_lowerednum_1mth * ts_std_dev(est_q_bps_lowerednum_1mth, 20), 22)', 4), ('ts_rank(- est_q_ebs_lowered_1wk * ts_std_dev(est_q_ebs_lowered_1wk, 20), 120)', 4), ('ts_rank(- vec_stddev(nws12_afterhsz_epsactual) * ts_std_dev(vec_stddev(nws12_afterhsz_epsactual), 20), 240)', 4), ('ts_std_dev(- mdl26_mstchg_pct_fq1_rvn_90 * ts_std_dev(mdl26_mstchg_pct_fq1_rvn_90, 20), 66)', 4), ('ts_mean(- fnd28_nddq1_value_04055q * ts_std_dev(fnd28_nddq1_value_04055q, 20), 66)', 4)], [('ts_returns(- anl15_cpsgics_ind_18_m_cos_dn * ts_std_dev(anl15_cpsgics_ind_18_m_cos_dn, 20), 5

In [18]:
len(pools)

223

## 4, simulate alphas

In [ ]:
# Simulate First Order
multi_simulate(pools, "SUBINDUSTRY", "USA", "TOP3000", 0)

b'{"user":{"id":"TJ91365"},"token":{"expiry":14400.0},"permissions":["CONSULTANT","MULTI_SIMULATION","PROD_ALPHAS","REFERRAL","VISUALIZATION","WORKDAY"]}'
pool 0 task 8 post done
Not complete : https://api.worldquantbrain.com/simulations/2ixX5a1Xi4HwcgAnXMTrhLI
Not complete : https://api.worldquantbrain.com/simulations/2Q2ueO9jK4Lq8Ok7S9tSsOC


## 5, Select alphas
go to web alphas penal to look for the number and date to track for next order improve

In [ ]:
## get promising alphas to improve in the next order
fo_tracker = get_alphas("01-01", "02-02", 1, 0.8, "USA", 1000, "track")

## 6, Next order improvement - Second Order
second order: ts_ops(field, days) -> group_ops(ts_ops(field, days), group)

In [ ]:
print(len(fo_tracker['next']))
print(len(fo_tracker['decay']))

fo_layer = prune(fo_tracker['next'] + fo_tracker['decay'], 'USA', 'mdl26', 5)
so_alpha_list = []
group_ops = group_ops = ["group_neutralize", "group_rank", "group_normalize", "group_scale", "group_zscore"]
for region, couples in fo_layer.items():
    for expr, decay in couples:
        for alpha in get_group_second_order_factory([expr], group_ops, region):
            so_alpha_list.append((alpha,decay))

print(len(so_alpha_list))
print(so_alpha_list[:3])

### Simulate second order

In [ ]:
so_pools = load_task_pool(so_alpha_list, 9, 10)
multi_simulate(so_pools, 'SUBINDUSTRY', 'USA', 'TOP3000', 0)

## Higher Order for improvement - Third Order
group_ops(ts_ops(field, days), group) -> trade_when(entre_event, group_ops(ts_ops(field, days), group), exit_event)

In [ ]:
## get promising alphas from second order to improve in the third order
so_tracker = get_alphas("01-01", "02-02", 1.4, 0.7, "USA", 360, "track")

print(len(so_tracker['next']))
print(len(so_tracker['decay']))

so_layer = prune(so_tracker['next'] + so_tracker['decay'], 'chn', 'mdl26', 5)
th_alpha_list = []
for region, couples in so_layer.items():
    for expr, decay in couples:
        for alpha in trade_when_factory("trade_when",expr,region):
            th_alpha_list.append((alpha,decay))
print(len(th_alpha_list))

### Simulate Third Order

In [ ]:
# Simulate third order
th_pools = load_task_pool(th_alpha_list, 9, 10)
multi_simulate(th_pools, 'SUBINDUSTRY', 'USA', 'TOP3000', 0)

## 7, Get submittable alphas
check_submission and view alphas

In [ ]:
# get submitable alphas to check submission
th_tracker = get_alphas("01-01", "02-02", 1.58, 1, "USA", 200, "submit")

In [ ]:
stone_bag = []
for alpha in th_tracker['next'] + th_tracker['decay']:
    stone_bag.append(alpha[0])
print(len(stone_bag))
gold_bag = []
check_submission(stone_bag, gold_bag, 0)

In [ ]:
# look date and metrics to locate alphas in the web
view_alphas(gold_bag)

## 8, fine-tune submittable alphas
neutralization, performance comparison, turnover, margin